In [1]:
#filename = "ml.py"
#exec(compile(open(filename, "rb").read(), filename, 'exec'))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import dateutil.parser as parser
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

# read the dataset into pandas dataframe
# df = pd.read_csv('./../downsampled_data', delim_whitespace=False).dropna()
df = pd.read_csv('./../downsampled_data.csv').dropna()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/trevorcarpenter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
df.columns

Index(['Unnamed: 0', 'video_id', 'title', 'publishedAt', 'channelId',
       'channelTitle', 'categoryId', 'duration', 'definition', 'trended_later',
       'time_retrieved', 'tags', 'thumbnail_link', 'ratings_disabled',
       'description', 'Channel_viewCount', 'Channel_subscriberCount',
       'Channel_hiddenSubscriberCount', 'Channel_videoCount', 'Channel_title',
       'Channel_description', 'Channel_publishedAt', 'Channel_country',
       'view_count_update_11_19_14', 'likes_update_11_19_14',
       'dislikes_update_11_19_14', 'comment_count_update_11_19_14',
       '11_19_14_update_timestamp', 'trending?', 'engagement_rate', 'INTL',
       'UNK', 'USA'],
      dtype='object')

In [5]:
df['categoryId'] = df['categoryId'].astype(np.float)

categories = pd.DataFrame(np.zeros((len(df), len(df['categoryId'].unique()))), columns = df['categoryId'].unique())
index = 0
for id in df['categoryId']:
    categories[id][index] = 1
    index += 1

In [6]:
# display the first five rows
retrieval_time = df['time_retrieved']
publish_time = df['publishedAt']
channel_publish_time = df['Channel_publishedAt']
retrieval_time_11_19_14 = df['11_19_14_update_timestamp']
columns_to_drop = ['Unnamed: 0', 'definition', 'categoryId', 'publishedAt', 'time_retrieved', 'Channel_title', '11_19_14_update_timestamp', 'Channel_publishedAt', 'video_id', 'channelId', 'thumbnail_link', 'Channel_country']
df = df.drop(columns_to_drop, axis = 1)

In [7]:
categories.columns

Float64Index([24.0, 25.0, 26.0, 17.0, 22.0, 20.0,  1.0, 10.0, 23.0, 28.0,  2.0,
              27.0, 19.0, 15.0, 29.0, 30.0],
             dtype='float64')

In [8]:
index = 0
for column in categories.columns:
    df.insert(df.shape[1], "Category_"+str(column), categories[column])
    index += 1

In [9]:
## time updates
import dateutil.parser as parser
age = []
age_update = []
channel_age = []
for i in df.index:
    channel_publish_time[i] = channel_publish_time[i].replace("\"", "")
    age.append(parser.isoparse(retrieval_time[i]) - parser.isoparse(publish_time[i]))
    age_update.append(parser.isoparse(retrieval_time_11_19_14[i]) - parser.isoparse(publish_time[i]))
    channel_age.append(parser.isoparse(channel_publish_time[i]) - parser.isoparse(publish_time[i]))
    if df['ratings_disabled'][i] == 'True':
        df['ratings_disabled'][i] = True
    elif df['ratings_disabled'][i] == 'False':
        df['ratings_disabled'][i] = False
        
    if df['Channel_hiddenSubscriberCount'][i] == 'True':
        df['Channel_hiddenSubscriberCount'][i] = True
    elif df['Channel_hiddenSubscriberCount'][i] == 'False':
        df['Channel_hiddenSubscriberCount'][i] = False
        
        
    if df['trended_later'][i] == 'True':
        df['trended_later'][i] = True
    elif df['trended_later'][i] == 'False':
        df['trended_later'][i] = False

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEn

In [10]:
## sentiment values

titles = df['title']
channel_title = df['channelTitle']
description = df['description']
channel_description = df['Channel_description']

title_sentiment_vals = []
channel_title_sentiment_vals = []
description_sentiment_vals = []
channel_description_sentiment_vals = []

sid = SentimentIntensityAnalyzer()
for sentence in titles:
    ss = sid.polarity_scores(str(sentence))
    title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_title:
    ss = sid.polarity_scores(str(sentence))
    channel_title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in description:
    ss = sid.polarity_scores(str(sentence))
    description_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_description:
    ss = sid.polarity_scores(str(sentence))
    channel_description_sentiment_vals.append(ss['pos']-ss['neg'])

In [11]:
df

,title,channelTitle,duration,trended_later,tags,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,...,Category_1.0,Category_10.0,Category_23.0,Category_28.0,Category_2.0,Category_27.0,Category_19.0,Category_15.0,Category_29.0,Category_30.0
0,Biden Victory Cold Open - SNL,Saturday Night Live,-0.0707230866960673,False,36.0,False,Joe Biden (Jim Carrey) and Donald Trump (Alec ...,0.819940,0.504608,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,How President Trump and the White House reacte...,CNBC Television,-0.0856798827419832,False,28.0,False,CNBC's Kayla Tausche joins The News with Shepa...,-0.274970,-0.350255,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Watch Live: President-Elect Joe Biden Addresse...,ABC News,0.09488158288091483,False,13.0,False,#Election2020 #Election #JoeBiden #PresidentEl...,0.650191,0.461214,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,I GOT PLASTIC SURGERY TO SEE HOW MY FRIENDS RE...,Alexa Rivera,-0.07104706061763949,False,0.0,False,"This was definitely such a fun prank, their re...",-0.258217,0.002973,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FC Barcelona 5 - 2 Real Betis - HIGHLIGHTS & G...,beIN SPORTS USA,-0.07288291283988188,False,12.0,False,Lionel Messi entró al segundo tiempo y resucit...,-0.330711,-0.407708,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,US Election: Recount likely as candidates figh...,Sky News,-0.08497793924524347,False,19.0,False,The fight for the White House has intensified ...,-0.192403,-0.192300,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,The Packers should trade for Dallas&#39; WR Am...,The Herd with Colin Cowherd,-0.07536671290526863,False,33.0,False,Trade rumors are swirling around Dallas Cowboy...,-0.269840,-0.361884,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877,Make You Mine This Season (Happiest Season),Tegan and Sara - Topic,-0.08692178277467658,False,1.0,False,Provided to YouTube by Facet/Warner Records Ma...,-0.340590,-0.440462,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
878,映画『ドクター・デスの遺産―BLACK FILE―』本編特別映像 2020年11月13日（金）公開,ワーナー ブラザース 公式チャンネル,-0.08395202182693154,False,35.0,False,綾野剛 × 北川景子 共演！ 警視庁捜査一課Ｎo.１コンビが挑む、 安楽死を手口にする連続殺...,-0.260114,-0.411267,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from sklearn import preprocessing

df['title'] = title_sentiment_vals
df['channelTitle'] = channel_title_sentiment_vals
df['description'] = description_sentiment_vals
df['Channel_description'] = channel_description_sentiment_vals

le = preprocessing.LabelEncoder()
df['trending?'] = le.fit_transform(df['trending?'])
df['ratings_disabled'] = le.fit_transform(df['ratings_disabled'])
df['Channel_hiddenSubscriberCount'] = le.fit_transform(df['Channel_hiddenSubscriberCount'])
pd.set_option('display.max_columns', None)
df = df.dropna()

In [13]:
# dummy values for now - get these interactively later...
import tensorflow as tf

best_model = False
if best_model:
    num_hidden_layers = 3
    num_hidden_layer_nodes = [20, 30, 20]
    train_ratio = .7
    hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
    optimizer = 'adam'
    learning_rate = .005
    loss = 'mean_squared_error'
    metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
    metrics_names = ["accuracy","recall","precision"]
    epochs = 300
    batch_size = 200
else:
    # build a custom model
    num_hidden_layers = 3
    num_hidden_layer_nodes = [20, 30, 20]
    train_ratio = .7
    hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
    optimizer = 'adam'
    learning_rate = .005
    loss = 'mean_squared_error'
    metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
    metrics_names = ["accuracy","recall","precision"]
    epochs = 300
    batch_size = 200

In [14]:
# Split the data into training and testing set by 70:30
ratio = 0.7
train, test = train_test_split(df, train_size=ratio, random_state=42)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [15]:
# separate data into x and y - just random y for now..
train_X = train.loc[:,train.columns != 'trending?']
train_Y = train['trending?']
test_X = test.loc[:,test.columns != 'trending?']
test_Y = test['trending?']

In [16]:
# build the logistic regression model - need clean data...
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(multi_class='ovr')
LR_model.fit(train_X, train_Y)

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(multi_class='ovr')

In [17]:
# confusion matrix
from sklearn.metrics import confusion_matrix

pred_Y = LR_model.predict(test_X);
confusion_matrix(test_Y, pred_Y)

array([[140,   4],
       [ 18,  48]])

In [18]:
# evaluate the accuracy of the LR model
accuracy = LR_model.score(test_X, test_Y)
print(accuracy)

0.8952380952380953


In [19]:
# build the ANN model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

ANN_model = keras.Sequential()

# add hidden layers
for i in range(num_hidden_layers):
    if i == 0:
        ANN_model.add(Dense(num_hidden_layer_nodes[i], input_dim = train_X.shape[1], activation=hidden_layer_activations[i]))
    ANN_model.add(Dense(num_hidden_layer_nodes[i], activation=hidden_layer_activations[i]))

# add output layers
ANN_model.add(Dense(1, activation=hidden_layer_activations[num_hidden_layers]))

ANN_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
ANN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                740       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 30)                630       
_________________________________________________________________
dense_3 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 2,431
Trainable params: 2,431
Non-trainable params: 0
_________________________________________________________________


In [20]:
# train the model
ANN_model.fit(train_X, train_Y, validation_data=(test_X, test_Y), epochs=epochs, batch_size=batch_size)

Train on 488 samples, validate on 210 samples
Epoch 1/300
488/488 [==============================] - 1s 2ms/sample - loss: 0.4136 - accuracy: 0.0000e+00 - recall: 1.0000 - precision: 0.2520 - val_loss: 0.3723 - val_accuracy: 0.0000e+00 - val_recall: 1.0000 - val_precision: 0.3143
Epoch 2/300
488/488 [==============================] - 0s 83us/sample - loss: 0.3935 - accuracy: 0.0000e+00 - recall: 1.0000 - precision: 0.2520 - val_loss: 0.3552 - val_accuracy: 0.0000e+00 - val_recall: 1.0000 - val_precision: 0.3143
Epoch 3/300
488/488 [==============================] - 0s 76us/sample - loss: 0.3739 - accuracy: 0.0000e+00 - recall: 1.0000 - precision: 0.2520 - val_loss: 0.3385 - val_accuracy: 0.0000e+00 - val_recall: 1.0000 - val_precision: 0.3143
Epoch 4/300
488/488 [==============================] - 0s 96us/sample - loss: 0.3539 - accuracy: 0.0000e+00 - recall: 1.0000 - precision: 0.2520 - val_loss: 0.3225 - val_accuracy: 0.0000e+00 - val_recall: 1.0000 - val_precision: 0.3143
Epoch 5/300

488/488 [==============================] - 0s 71us/sample - loss: 0.1884 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2182 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 34/300
488/488 [==============================] - 0s 58us/sample - loss: 0.1884 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2184 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 35/300
488/488 [==============================] - 0s 71us/sample - loss: 0.1883 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2184 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 36/300
488/488 [==============================] - 0s 67us/sample - loss: 0.1883 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2184 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
E

488/488 [==============================] - 0s 122us/sample - loss: 0.1866 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2163 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 66/300
488/488 [==============================] - 0s 106us/sample - loss: 0.1865 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2160 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 67/300
488/488 [==============================] - 0s 57us/sample - loss: 0.1864 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2159 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 68/300
488/488 [==============================] - 0s 71us/sample - loss: 0.1863 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2157 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

488/488 [==============================] - 0s 59us/sample - loss: 0.1787 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2037 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 98/300
488/488 [==============================] - 0s 78us/sample - loss: 0.1782 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2030 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 99/300
488/488 [==============================] - 0s 130us/sample - loss: 0.1777 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2023 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 100/300
488/488 [==============================] - 0s 55us/sample - loss: 0.1771 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2017 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

488/488 [==============================] - 0s 67us/sample - loss: 0.1463 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.1574 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 130/300
488/488 [==============================] - 0s 145us/sample - loss: 0.1447 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.1554 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 131/300
488/488 [==============================] - 0s 84us/sample - loss: 0.1432 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.1533 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 132/300
488/488 [==============================] - 0s 68us/sample - loss: 0.1417 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.1513 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+

Epoch 163/300
488/488 [==============================] - 0s 67us/sample - loss: 0.1007 - accuracy: 0.0000e+00 - recall: 0.7317 - precision: 0.7759 - val_loss: 0.1035 - val_accuracy: 0.0000e+00 - val_recall: 0.8030 - val_precision: 0.8689
Epoch 164/300
488/488 [==============================] - 0s 114us/sample - loss: 0.0999 - accuracy: 0.0000e+00 - recall: 0.7317 - precision: 0.7759 - val_loss: 0.1024 - val_accuracy: 0.0000e+00 - val_recall: 0.8030 - val_precision: 0.8689
Epoch 165/300
488/488 [==============================] - 0s 97us/sample - loss: 0.0991 - accuracy: 0.0000e+00 - recall: 0.7317 - precision: 0.7759 - val_loss: 0.1014 - val_accuracy: 0.0000e+00 - val_recall: 0.8030 - val_precision: 0.8689
Epoch 166/300
488/488 [==============================] - 0s 70us/sample - loss: 0.0985 - accuracy: 0.0000e+00 - recall: 0.7236 - precision: 0.7739 - val_loss: 0.1005 - val_accuracy: 0.0000e+00 - val_recall: 0.8030 - val_precision: 0.8689
Epoch 167/300
488/488 [========================

488/488 [==============================] - 0s 101us/sample - loss: 0.0849 - accuracy: 0.0000e+00 - recall: 0.8293 - precision: 0.7556 - val_loss: 0.0908 - val_accuracy: 0.0000e+00 - val_recall: 0.8333 - val_precision: 0.7971
Epoch 198/300
488/488 [==============================] - 0s 48us/sample - loss: 0.0845 - accuracy: 0.0000e+00 - recall: 0.8293 - precision: 0.7556 - val_loss: 0.0900 - val_accuracy: 0.0000e+00 - val_recall: 0.8333 - val_precision: 0.8209
Epoch 199/300
488/488 [==============================] - 0s 59us/sample - loss: 0.0844 - accuracy: 0.0000e+00 - recall: 0.8293 - precision: 0.7556 - val_loss: 0.0893 - val_accuracy: 0.0000e+00 - val_recall: 0.8182 - val_precision: 0.8182
Epoch 200/300
488/488 [==============================] - 0s 67us/sample - loss: 0.0842 - accuracy: 0.0000e+00 - recall: 0.8293 - precision: 0.7612 - val_loss: 0.0893 - val_accuracy: 0.0000e+00 - val_recall: 0.8333 - val_precision: 0.8209
Epoch 201/300
488/488 [==============================] - 0s 5

488/488 [==============================] - 0s 98us/sample - loss: 0.0798 - accuracy: 0.0000e+00 - recall: 0.8618 - precision: 0.7737 - val_loss: 0.0886 - val_accuracy: 0.0000e+00 - val_recall: 0.8636 - val_precision: 0.7917
Epoch 232/300
488/488 [==============================] - 0s 82us/sample - loss: 0.0797 - accuracy: 0.0000e+00 - recall: 0.8618 - precision: 0.7737 - val_loss: 0.0893 - val_accuracy: 0.0000e+00 - val_recall: 0.8636 - val_precision: 0.7917
Epoch 233/300
488/488 [==============================] - 0s 52us/sample - loss: 0.0795 - accuracy: 0.0000e+00 - recall: 0.8780 - precision: 0.7714 - val_loss: 0.0893 - val_accuracy: 0.0000e+00 - val_recall: 0.8636 - val_precision: 0.7917
Epoch 234/300
488/488 [==============================] - 0s 51us/sample - loss: 0.0794 - accuracy: 0.0000e+00 - recall: 0.8699 - precision: 0.7754 - val_loss: 0.0889 - val_accuracy: 0.0000e+00 - val_recall: 0.8636 - val_precision: 0.7917
Epoch 235/300
488/488 [==============================] - 0s 95

488/488 [==============================] - 0s 127us/sample - loss: 0.0773 - accuracy: 0.0000e+00 - recall: 0.8862 - precision: 0.7786 - val_loss: 0.0884 - val_accuracy: 0.0000e+00 - val_recall: 0.8788 - val_precision: 0.7945
Epoch 266/300
488/488 [==============================] - 0s 69us/sample - loss: 0.0773 - accuracy: 0.0000e+00 - recall: 0.8862 - precision: 0.7786 - val_loss: 0.0882 - val_accuracy: 0.0000e+00 - val_recall: 0.8636 - val_precision: 0.7917
Epoch 267/300
488/488 [==============================] - 0s 71us/sample - loss: 0.0772 - accuracy: 0.0000e+00 - recall: 0.8780 - precision: 0.7826 - val_loss: 0.0884 - val_accuracy: 0.0000e+00 - val_recall: 0.8788 - val_precision: 0.7945
Epoch 268/300
488/488 [==============================] - 0s 51us/sample - loss: 0.0771 - accuracy: 0.0000e+00 - recall: 0.8862 - precision: 0.7786 - val_loss: 0.0889 - val_accuracy: 0.0000e+00 - val_recall: 0.8788 - val_precision: 0.7838
Epoch 269/300
488/488 [==============================] - 0s 6

488/488 [==============================] - 0s 67us/sample - loss: 0.0763 - accuracy: 0.0000e+00 - recall: 0.8943 - precision: 0.7857 - val_loss: 0.0910 - val_accuracy: 0.0000e+00 - val_recall: 0.8788 - val_precision: 0.7838
Epoch 300/300
488/488 [==============================] - 0s 62us/sample - loss: 0.0762 - accuracy: 0.0000e+00 - recall: 0.8943 - precision: 0.7857 - val_loss: 0.0898 - val_accuracy: 0.0000e+00 - val_recall: 0.8788 - val_precision: 0.7838


In [21]:
# confusion matrix
pred_Y = ANN_model.predict_classes(test_X);
confusion_matrix(test_Y, pred_Y)

array([[128,  16],
       [  8,  58]])

In [22]:
# report evaluation metrics 
evaluated_metrics = ANN_model.evaluate(test_X, test_Y)
for i in range(len(metrics)):
    print(metrics_names[i] + ": %.2f" % evaluated_metrics[i])

210/210 [==============================] - 0s 304us/sample - loss: 0.0898 - accuracy: 0.0000e+00 - recall: 0.8788 - precision: 0.7838
accuracy: 0.09
recall: 0.00
precision: 0.88


SyntaxError: invalid syntax (<ipython-input-25-f0fc12405069>, line 1)